In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/electricity-theft-detection/visualization.csv
/kaggle/input/electricity-theft-detection/Validation_SansFLAG.csv
/kaggle/input/electricity-theft-detection/Validation.csv
/kaggle/input/electricity-theft-detection/preprocessedR95.csv


In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[3]:


import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
import datetime


# In[4]:


df = pd.read_csv('/kaggle/input/electricity-theft-detection/preprocessedR95.csv', header=0, parse_dates=[0], index_col=0, squeeze=True) 

df


# # Visualiser la consommation d'éléctricité pour un client avec un FLAG = 1, i.e : with FRAUD

# In[19]:


# Client AVEC FRAUD
# Transposer les données
df_transposed = df.set_index('CONS_NO').T

# Sélectionner les données à tracer (par exemple, CONS_NO='4B75AC4F2D8434CFF62DB64D0BB43103')
selected_data = df_transposed['4B75AC4F2D8434CFF62DB64D0BB43103']

# Convertir les index en datetime si nécessaire
selected_data.index = pd.to_datetime(selected_data.index)

# Tracer les données
plt.figure(figsize=(15, 8))
plt.plot(selected_data.index, selected_data.values)
plt.xlabel('Date')
plt.ylabel('Consommation')
plt.title('Tracé des données pour CONS_NO="4B75AC4F2D8434CFF62DB64D0BB43103"')
plt.show()


# Est-ce un modèle additif ?

# In[20]:


import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Convertir l'index en DatetimeIndex avec une fréquence quotidienne
selected_data.index = pd.date_range(start=selected_data.index[0], periods=len(selected_data), freq='D')

# Appliquer seasonal_decompose avec l'index mis à jour
TSA = seasonal_decompose(selected_data).plot()


# Est-ce un modèle multiplicatif ?

# In[24]:


import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Assurez-vous que l'index est un DatetimeIndex
selected_data.index = pd.to_datetime(selected_data.index)

# Appliquer seasonal_decompose avec le modèle multiplicatif
TSA = seasonal_decompose(selected_data, model='multiplicative')
TSA.plot()
plt.show()


# La série est-elle stationnaire ? (Test de ADFuller)

# In[26]:


from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(selected_data.values)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])


# p-value < 0,05 ==> Série stationnaire 

# ACF/PACF

# Sans différenciation

# In[28]:


# Client AVEC FRAUD
# Transposer les données
df_transposed = df.set_index('CONS_NO').T

# Sélectionner les données à tracer (par exemple, CONS_NO='4B75AC4F2D8434CFF62DB64D0BB43103')
selected_data = df_transposed['4B75AC4F2D8434CFF62DB64D0BB43103']

# Convertir les index en datetime si nécessaire
selected_data.index = pd.to_datetime(selected_data.index)

# Tracer les données
plt.figure(figsize=(15, 8))
plt.plot(selected_data.index, selected_data.values)
plt.xlabel('Date')
plt.ylabel('Consommation')
plt.title('Tracé des données pour CONS_NO="4B75AC4F2D8434CFF62DB64D0BB43103"')
plt.show()


# In[27]:


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,7))

plot_acf(selected_data, lags = 36, ax=ax1)
plot_pacf(selected_data, lags = 36, ax=ax2)
plt.show()


# Avec différenciation d'ordre 1

# In[31]:


plt.figure(figsize=(10,5))
plt.plot(selected_data.diff());


# In[33]:


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,7))

plot_acf(selected_data.diff().dropna(), lags = 36, ax=ax1)
plot_pacf(selected_data.diff().dropna(), lags = 36, ax=ax2)
plt.show()


# Avec différenciation d'ordre 12

# # Visualiser la consommation d'éléctricité pour un client avec un FLAG = 0, i.e : WITHOUT FRAUD

# In[5]:


# Client NON FRAUD
# Transposer les données
df_transposed = df.set_index('CONS_NO').T

# Sélectionner les données à tracer (par exemple, CONS_NO='F3406636BAD1E6E0826E8EDDC9A1BF00')
selected_data_NOT_FRAUD = df_transposed['F3406636BAD1E6E0826E8EDDC9A1BF00']

# Convertir les index en datetime si nécessaire
selected_data_NOT_FRAUD.index = pd.to_datetime(selected_data_NOT_FRAUD.index)

# Tracer les données
plt.figure(figsize=(15, 8))
plt.plot(selected_data_NOT_FRAUD.index, selected_data_NOT_FRAUD.values)
plt.xlabel('Date')
plt.ylabel('Consommation')
plt.title('Tracé des données pour CONS_NO="F3406636BAD1E6E0826E8EDDC9A1BF00"')
plt.show()


# In[6]:


from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(selected_data_NOT_FRAUD.values)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])


# # p-value > 0.05 on accepte H0 ==> Série n'est pas stationnaire 

# Est-ce un modèle additif ?

# In[7]:


import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Convertir l'index en DatetimeIndex avec une fréquence quotidienne
selected_data_NOT_FRAUD.index = pd.date_range(start=selected_data_NOT_FRAUD.index[0], periods=len(selected_data_NOT_FRAUD), freq='D')

# Appliquer seasonal_decompose avec l'index mis à jour
TSA = seasonal_decompose(selected_data_NOT_FRAUD).plot()


# Est-ce un modèle multiplicatif ?

# In[8]:


import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Assurez-vous que l'index est un DatetimeIndex
selected_data_NOT_FRAUD.index = pd.to_datetime(selected_data_NOT_FRAUD.index)

# Appliquer seasonal_decompose avec le modèle multiplicatif
TSA = seasonal_decompose(selected_data_NOT_FRAUD, model='multiplicative')
TSA.plot()
plt.show()


# In[9]:


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,7))

plot_acf(selected_data_NOT_FRAUD, lags = 36, ax=ax1)
plot_pacf(selected_data_NOT_FRAUD, lags = 36, ax=ax2)
plt.show()


# Avec différenciation d'ordre 1

# In[8]:


plt.figure(figsize=(10,5))
plt.plot(selected_data_NOT_FRAUD.diff());


# In[11]:


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,7))

plot_acf(selected_data_NOT_FRAUD.diff().dropna(), lags = 36, ax=ax1)
plot_pacf(selected_data_NOT_FRAUD.diff().dropna(), lags = 36, ax=ax2)
plt.show()


# In[ ]:




